In [1]:
import model_based_mvpa as mbmvpa
from model_based_mvpa.preprocessing.bids import *
from model_based_mvpa.preprocessing.events import *
from model_based_mvpa.utils.func import *

In [ ]:
root = '/data2/project_model_based_fmri/ds000005/'

In [ ]:
X, masked_data = bids_preprocess(root, single_file=True, smoothing_fwhm=None, zoom=(2, 2, 2), ncore=os.cpu_count())

In [ ]:
funcs = [adjust_event_columns, calculate_modulation] # todo: Modifying to receive user's utility function

In [ ]:
a, b, y = preprocess_events(root, dm_model='ra_prospect', funcs=funcs, ncore=4)

In [ ]:
X, y = prepare_data(X, y)

## model